In [3]:
import pandas as pd
import keras
import numpy as np
import pickle as pk
from io import StringIO
from scipy import spatial
from sklearn.feature_selection import chi2

In [7]:
data_url = "./divar_posts_dataset.csv"

In [5]:
data = pd.read_csv(data_url)

In [8]:
a = data[data['cat1'] == 'electronic-devices'][data['brand'] == data['brand']]['brand'].unique()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [9]:
df = data[data['cat1'] == 'electronic-devices'][data['brand'] == data['brand']]
col = ['brand', 'desc']
df = df[col]
df
df['brand_id'] = df['brand'].factorize()[0]
brand_id_df = df[['brand', 'brand_id']].drop_duplicates().sort_values('brand_id')
brand_to_id = dict(brand_id_df.values)
id_to_brand = dict(brand_id_df[['brand_id', 'brand']].values)
df.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,brand,desc,brand_id
11,Nokia::نوکیا,سلام.یه گوشیه6303سالم که فقط دوتا خط کوچیک رو ...,0
19,Apple::اپل,درحد نو سالم اصلى بدون ضربه مهلت تست میدم,1
24,Samsung::سامسونگ,گوشى بسیار بسیار تمیز و فقط سه هفته کارکرده و ...,2
26,Apple::اپل,گلس پشت و رو .کارت اپل ای دی. لوازم جانبی اصلی...,1
64,Samsung::سامسونگ,کاملا تمیز و بدون حتی 1 خط و خش\nبه همراه گلاس...,2


In [10]:
testing = pd.read_csv('sample_mobile_data_1000.csv')
testing.title.values[200]

'A7 2016 بسیار نو و سالم'

In [14]:
testing

,archive_by_user,city,created_at,image_count,desc,image_count.1,mileage,platform,price,title,type,year
0,False,Tehran,Thursday 07PM,0,گوشی سالم ونو,0,NaN,mobile,250000,گوشی آیفون مدلs7,NaN,NaN
1,True,Shiraz,Thursday 03PM,0,سلام \n4s 64gig LLAسفید \nعکس نذاشتم چون یه نق...,0,NaN,mobile,550000,آیفون 4s 64gig,NaN,NaN
2,True,Shiraz,Wednesday 01PM,3,با سلام \n\nفروش گوشی اس6 سفید 32 گیگ\n\nفقط گ...,3,NaN,mobile,550000,گوشی موبایل سامسونگ اس6 لایو دمو,NaN,NaN
3,False,Tabriz,Sunday 07PM,3,گوشی در حد آک و بدون تعمیر.,3,NaN,mobile,90000,فروش گوشی عطیقه نوکیا ۱۱۰۰.,NaN,NaN
4,True,Ahvaz,Friday 02PM,2,گوشی مدل پایین تمیزه,2,NaN,mobile,300000,گوشی سامسونگ مدل پایین,NaN,NaN
5,True,Shiraz,Saturday 06PM,3,خیلی خیلی تمیز هست واز لحاظ باطری که حرف نداره...,3,NaN,mobile,1000000,iPhone5s32گری,NaN,NaN
6,True,Shiraz,Thursday 06AM,1,به صورت نقد و اقساط.با تشکر از مدیریت دیوار,1,NaN,mobile,500000,گوشی های جدید جی ال ایکس با گارانتی,NaN,NaN
7,False,Tehran,Sunday 09PM,3,دو سیم کارت.یک سال و نیم کارکرده خیلی تمیز و س...,3,NaN,mobile,520000,گوشی سامسونگ e5 در حد نو,NaN,NaN
8,False,Karaj,Friday 12PM,2,سالم بدون ایراد از اولم دست خودم بوده,2,NaN,mobile,250000,گوشی هواوی g610,NaN,NaN
9,True,Tehran,Tuesday 06AM,1,سالم به شرط در‌lla,1,NaN,mobile,1300000,ایفون۶‌گلد‌۱۶‌گیگ,NaN,NaN


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', ngram_range=(1, 2))

In [18]:
features = tfidf.fit_transform(df.desc).toarray()
labels = df.brand_id
features.shape

(70441, 37181)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df['desc'], df['brand'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [23]:
print(clf.predict(count_vect.transform([testing.title.values[975]])))
print(clf.predict(count_vect.transform(['یک گوشی htc one دارم'])))

['Huawei::هوآوی']
['HTC::اچ\u200cتی\u200cسی']
